<center>
  <font size=6><b>Electronic Thesis and Dissertations Automatic Guidelines Verification
</b></font>

   Mubanga Chibesa 2020031345@student.unza.zm<sup>1</sup> Albertina Mooka 2020019299@student.unza.zm<sup>2</sup> Gift Muwele 2020014203@student.unza.zm<sup>2</sup> and Lwiime Shansonga 2020002353@student.unza.zm <sup>4</sup>

  <sup>1</sup> Department of Library and Information Science, University of Zambia, P.O Box 32379, Lusaka, Zambia</br>

  Supervisor: Dr.Lighton Phiri lighton.phiri@unza.zm
  <br/>

  August 2024
</center>

<justify>
Higher Education Institutions worldwide enforce guidelines and academic approaches to ensure scholarly integrity and adherence to academic standards.The University of Zambia, is not an exception. like most HEIs it offers  training to postgraduate students and one of the key aspects of postgraduate training is the production of an Electronic Thesis and Dissertation manuscript. The Directorate of Research and Graduate Studies (DRGS) of the University of Zambia provides guidelines which stipulate how ETD’s should be formatted. However, the process of  checking for conformance is a manual and tedious process, resulting in submission of inconsistently formatted manuscripts in the Institutional Repository (UNZA IR).
To address this challenge, we embarked on a  project to implement a tool that will automate the process of checking ETD’s compliance against established postgraduate guidelines, leveraging data mining techniques to perform these tasks. More specifically Document Layout Analysis (DLA) as the core approach used in  the implementation. The tool flags off portions of ETD manuscripts that do not  conform to stipulated guidelines.
Using a mixed methods approach, document analysis was employed to understand the postgraduate guidelines stipulated in the “Regulations and Guidelines for Postgraduate Studies” guidelines document; content analysis was  used to randomly sampled ETDs in order to experimentally determine the extent of the problem and, finally interviews and questionairs were used to find out the challenges that postgraduates, Alumni and Lecturers encounter during the process.
Upon successful completion of this project, we anticipate a reduced workload associated with the process of manually checking if manuscripts conform to postgraduate guidelines, freeing up time and resources to handle other academic tasks like attending more to the content of the manuscripts. Not only that, academic integrity is consequently going to be improved through the standardisation of ETD formats and this will add to the university’s reputation and credibility.
</justify>




**Proposed Solution**

 To implement a tool that will automate the process of checking the compliance of ETDs to postgraduate guidelines and consequently enhance the compliance levels of ETDs published on the University of Zambia's institutional repository thereafter. The tool will have a focus on improving the efficiency of the document  review process by employing data mining techniques, specifically Document Layout Analysis (DLA) Strategies in the implementation of this pipeline. The key feature of this tool is evaluating the layout of ETDs against institutional guidelines and flagging off portions of these manuscripts that do not conform to the guidelines

 This Notebook outlines how data mining techniques were utilised to flag fo portions of manuscripts that do not conform to the postgraduate guidelines.

**Methodology**

**Packages and Libraries Imports**

In [ ]:
# @title Libraries and Packages Imports
!apt-get install poppler-utils
!pip install --upgrade deepdoctection
!pip install deepdoctection
!pip install Tensorflow
!pip install pytorch
!apt-get install tesseract-ocr
!pip install pdfplumber
!pip install pdf2image

import deepdoctection as dd
from pdf2image import convert_from_path
from pathlib import Path
from matplotlib import pyplot as plt
from pathlib import Path
from deepdoctection import get_dd_analyzer
from google.colab import files
uploaded = files.upload()
import json
analyzer = dd.get_dd_analyzer(config_overwrite=["LANGUAGE='eng'"])








An API that will be linking the frontend and backend using flask.

In [ ]:
!pip install flask

In [ ]:
from flask import Flask, request, jsonify
@app.after_request
def after_request(response):
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization')
    response.headers.add('Access-Control-Allow-Methods', 'GET,PUT,POST,DELETE')
    return response
# Initialize Flask app
app = Flask(__name__)

# Example GET endpoint
@app.route('/api/upload', methods=['GET'])
def handle-upload():
    return jsonify(message='Hello from the backend!')

# Example POST endpoint
@app.route('/api/data', methods=['POST'])
def receive_data():
    data = request.json
    name = data.get('name', 'Unknown')
    return jsonify(message=f'Hello, {name}!')

# New endpoint to handle file uploads
@app.route('/api/upload', methods=['POST'])
def upload_file():
    # Check if a file is part of the request
    if 'file' not in request.files:
        return jsonify(message='No file part in the request'), 400

    file = request.files['file']

    # Check if a file was submitted
    if file.filename == '':
        return jsonify(message='No file selected'), 400

    # Save file (optional)
    # file.save(f'/path/to/save/{file.filename}')

    # Perform any processing needed with the file
    file_content = file.read()  # Example: read file content

    return jsonify(message=f'File {file.filename} uploaded successfully!', content=str(file_content))

# Run the Flask app
if __name__ == '__main__':
    app.run(port=3000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:3000
INFO:werkzeug:Press CTRL+C to quit


**Document Format Compliance Verification**

In [ ]:
pdf_file = next(iter(uploaded)) # Access uploaded pdf file

# Set PDF file path
pdf_path = Path(r"ETD-ED1.pdf")

# Convert PDF to a list of images (one per page)
pages = convert_from_path(pdf_path, dpi=300)

# Check if the document pages are less than 150
if len(pages) >= 150:
    print("The document has 150 or more pages.")
else:
    print("The document has fewer than 150 pages.")

# Display the first page or any other page by indexing into `pages`)
plt.figure(figsize=(25, 17))
plt.axis('off')
plt.imshow(pages[0])  # pages[0] corresponds to the first page of the PDF
plt.show()

# Process each page image using the analyzer
results = []
for i, page_image in enumerate(pages):
    # Convert the PIL image to a format compatible with deepdoctection
    page_image_path = Path(f"page_{i}.png")
    page_image.save(page_image_path)  # file save to unique path for each page

    # Analyze the image
    analysis_result = analyzer.analyze(path=pdf_path, page_number=1+1)
    results.append(analysis_result)
    print(f"Analysis result for page {i}:", analysis_result)

    # Check font size for each page (assumed field in analysis_result)
    if 'font_size' in analysis_result:
        font_size = analysis_result['font_size']
        print(f"Page {i} font size:", font_size)

        if font_size != 12:
            print(f"Warning: Page {i} does not have the correct font size. Expected 12pt.")
    else:
        print(f"Font size information not available for page {i}.")

#Check for abstract content
    if 'abstract' in analysis_result:
        abstract_content = analysis_result['abstract']
        abstract_length_chars = len(abstract_content)  # Count the number of characters

        # If 'abstract' is structured as a list (one entry per page), count its length
        if isinstance(abstract_content, list):
            abstract_length_pages = len(abstract_content)
            abstract_text_combined = " ".join(abstract_content)  # Combine all parts for character count
            abstract_length_chars = len(abstract_text_combined)
        else:
            abstract_length_pages = 1  # Assume single page if not a list

        print(f"Abstract length: {abstract_length_pages} page(s), {abstract_length_chars} characters")

        # Check if the abstract is exactly one page or 300 characters long
        if abstract_length_pages == 1 or abstract_length_chars == 300:
            print("The abstract meets the requirement of being one page long or exactly 300 characters.")
        else:
            print("Warning: The abstract does not meet the requirement of being one page long or exactly 300 characters.")
    else:
        print(f"Abstract content not found in analysis result for page {i}.")


# Save results as a JSON file
with open("analysis_results.json", "w") as f:
    json.dump(results, f, indent=4)
